In [2]:
import os

In [3]:
google_project_id = os.getenv('GOOGLE_PROJECT_ID')
google_biquery_dataset = os.getenv('GOOGLE_BIGQUERY_DATASET')
google_cloud_storage_id = os.getenv('GOOGLE_CLOUD_STORAGE_ID')

In [4]:
print(f"proj ID {google_project_id} withcloud storage id at {google_cloud_storage_id} and bigquery dataset being {google_biquery_dataset}")

proj ID validation-project-de-zoomcamp withcloud storage id at million_song_data_lake_validation-project-de-zoomcamp and bigquery dataset being million_song


In [5]:
!wget https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop3-latest.jar -O /spark/gcs-connector-hadoop3-latest.jar
!wget https://storage.googleapis.com/spark-lib/bigquery/spark-bigquery-latest_2.12.jar -O /spark/spark-bigquery-latest_2.12.jar

--2022-03-27 09:57:46--  https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop3-latest.jar
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.10.128, 142.251.12.128, 172.217.194.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.10.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31607894 (30M) [application/java-archive]
Saving to: ‘/spark/gcs-connector-hadoop3-latest.jar’

/spark/gcs-connecto 100%[===================>]  30.14M  18.4MB/s    in 1.6s    

2022-03-27 09:57:48 (18.4 MB/s) - ‘/spark/gcs-connector-hadoop3-latest.jar’ saved [31607894/31607894]

--2022-03-27 09:57:48--  https://storage.googleapis.com/spark-lib/bigquery/spark-bigquery-latest_2.12.jar
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.10.128, 142.251.12.128, 172.217.194.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.10.128|:443... connected.
HTTP request sent, awaiting respons

In [6]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('test') \
    .set("spark.jars", "/spark/gcs-connector-hadoop3-latest.jar,/spark/spark-bigquery-latest_2.12.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", "/.google/google_credentials.json")

sc = SparkContext(conf=conf)

sc._jsc.hadoopConfiguration().set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
sc._jsc.hadoopConfiguration().set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
sc._jsc.hadoopConfiguration().set("fs.gs.auth.service.account.json.keyfile", "/.google/google_credentials.json")
sc._jsc.hadoopConfiguration().set("fs.gs.auth.service.account.enable", "true")

22/03/27 09:57:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [7]:
spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

In [8]:
df_million_song = spark.read \
    .option("header", "true") \
    .parquet(f"gs://{google_cloud_storage_id}/raw/millionsongsubset.parquet")

df_million_song = df_million_song.repartition(4)

df_million_song.show()

22/03/27 09:57:59 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: gs://million_song_data_lake_validation-project-de-zoomcamp/raw/millionsongsubset.parquet.
java.io.IOException: Error accessing gs://million_song_data_lake_validation-project-de-zoomcamp/raw/millionsongsubset.parquet
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.gcsio.GoogleCloudStorageImpl.getObject(GoogleCloudStorageImpl.java:2155)
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.gcsio.GoogleCloudStorageImpl.getItemInfo(GoogleCloudStorageImpl.java:2043)
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.gcsio.GoogleCloudStorageFileSystem.getFileInfoInternal(GoogleCloudStorageFileSystem.java:1091)
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.gcsio.GoogleCloudStorageFileSystem.getFileInfo(GoogleCloudStorageFileSystem.java:1065)
	at com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystemBase

Py4JJavaError: An error occurred while calling o143.parquet.
: java.io.IOException: Error accessing gs://million_song_data_lake_validation-project-de-zoomcamp/raw/millionsongsubset.parquet
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.gcsio.GoogleCloudStorageImpl.getObject(GoogleCloudStorageImpl.java:2155)
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.gcsio.GoogleCloudStorageImpl.getItemInfo(GoogleCloudStorageImpl.java:2043)
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.gcsio.GoogleCloudStorageFileSystem.getFileInfoInternal(GoogleCloudStorageFileSystem.java:1091)
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.gcsio.GoogleCloudStorageFileSystem.getFileInfo(GoogleCloudStorageFileSystem.java:1065)
	at com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystemBase.getFileStatus(GoogleHadoopFileSystemBase.java:955)
	at org.apache.hadoop.fs.FileSystem.exists(FileSystem.java:1760)
	at org.apache.spark.sql.execution.

# What each column represent ?

### analysis_sample_rate
- The sampling frequency or sampling rate, fs, is the average number of samples obtained in one second, thus fs = 1/T. **Int**

### artist_7digitalid
- The artist 7digital id -> https://us.7digital.com/artist/and-id **Int*

### artist_familiarity
- Familiarity is an indication of how well known the artist is. **Float**

### artist_hotttnesss
- Hotness is an indication of how much buzz the artist is getting  right now **Float**

### artist_id
- Id of artist in millionsong dataset **String**

### artist_latitude 	artist_location 	artist_longitude 	
- If present Longitue / Latitude as **Float** and location as **String**

### artist_mbid
- Id from musicBrainz for the artist **String**

###  artist_mbtags     artist_mbtags_count
- Array of tags representing the artist, as well as the count of those tags from MusicBrainz **List[String] && List[Int]**

### artist_name
- Name of artist **String**

### artist_playmeid
- Playme Id of the artist

### artist_terms       artist_terms_freq      artist_terms_weight
- Array of tags representing the artist, as well as the frequency and weight of those tags from The Echo Nest **List[String] && List[Float]  List[Float]**

### audio_md5
- hash code of the audio used for the analysis by The Echo Nest **Sring**

### bars_confidence
- confidence value (between 0 and 1) associated with each bar by The Echo Nest **List[Float]**

### bars_start
- start time of each bar according to The Echo Nest, this song has 99 bars **List[Float]**

### beats_confidence 
- confidence value (between 0 and 1) associated with each beat by The Echo Nest **List[Float]**

### beats_start
- start time of each beat according to The Echo Nest**List[Float]**

### danceability 
- danceability measure of this song according to The Echo Nest (between 0 and 1, 0 => not analyzed) **Float**

### duration
- Duration of track in seconds **Float**

### end_of_fade_in
- time of the end of the fade in, at the beginning of the song, according to The Echo Nest **Float**

### energy
- 0.0 everywhere

### key
- estimation of the key the song is in by The Echo Nest **Int**

### key_confidence
- confidence of the key estimation **Float**

### loudness
- general loudness of the track **Float**

### mode
- estimation of the mode (major or minor) the song is in by The Echo Nest **Int**

### mode_confidence
- confidence of the mode estimation **Float**

### release
- album name from which the track was taken, some songs / tracks can come from many albums, we give only one **String**

### release_7digitalid
- the ID of the release (album) on the service 7digital.com **String**

### sections_confidence
- confidence value (between 0 and 1) associated with each section by The Echo Nest **List[Float]**

### sections_start
- start time of each section according to The Echo Nest **List[Float]**

### segments_confidence
- confidence value (between 0 and 1) associated with each segment by The Echo Nest **List[Float]**

### segments_loudness_max
- max loudness during each segment **List[Float]**

### segments_loudness_max_time
-  time of the max loudness during each segment **List[Float]**
 
### segments_loudness_start
- loudness at the beginning of each segment **List[Float]**
 
### segments_start
- start time of each segment (~ musical event, or onset) according to The Echo Nest **List[Float]**
 
### similar_artists
- a list of 100 artists (their Echo Nest ID) similar to Rick Astley according to The Echo Nest **List[String]**

### song_hotttnesss
- according to The Echo Nest, when downloaded (in December 2010), this song had a 'hotttnesss' of this value (on a scale of 0 and 1) **Float**

### song_id
-  The Echo Nest song ID, note that a song can be associated with many tracks (with very slight audio differences) **String**

### start_of_fade_out
-  start time of the fade out, in seconds, at the end of the song, according to The Echo Nest **Float**

### tatums_confidence
-  confidence value (between 0 and 1) associated with each tatum by The Echo Nest **List[Float]**

### tatums_start
- start time of each tatum according to The Echo Nest **List[Float]**

### tempo
- Tempo of the song **Float**

### time_signature
- time signature of the song according to The Echo Nest, i.e. usual number of beats per bar **Int**

### time_signature_confidence
- confidence of the time signature estimation **Float**

### title
- Title of Song **String**

### track_7digitalid
- the ID of this song on the service 7digital.com **String**

### track_id
- The Echo Nest ID of this particular track on which the analysis was done **String**

### year
- year when this song was released, according to musicbrainz.org **Int**

In [ ]:
# let's look at the describe
df_million_song.printSchema()

In [ ]:
df_million_song.count()

In [ ]:
## Decision for storage efficiency

## We will parition by year, even if more than half data is at year 0 (5320)
df_million_song.groupBy('year').count().show(100)

In [ ]:
df_million_song.groupBy('key').count().show()

In [ ]:
df_million_song.groupBy('mode').count().show()

## We transform the tempo to int to be able to leverage cluster of bigquery

In [ ]:
from pyspark.sql.types import IntegerType
df_million_song = df_million_song.withColumn('tempo_int', (df_million_song.tempo).cast(IntegerType())).drop('tempo')
df_million_song.groupBy('tempo_int').count().show(100)

In [ ]:
df_to_save = df_million_song.drop('bars_confidence','bars_start','beats_confidence', 'beats_start', 'sections_confidence', 'sections_start', 'segments_confidence', 'segments_loudness_max', 'segments_loudness_max_time', 'segments_loudness_start', 'segments_start')

In [16]:
record = df_to_save.sample(False, 0.1, seed=0).limit(5)

In [17]:
record.write \
        .format("bigquery") \
        .option("temporaryGcsBucket",google_cloud_storage_id) \
        .option('table', f"{google_biquery_dataset}.data") \
        .save()

In [19]:

from google.cloud import bigquery

client = bigquery.Client()

dataset_ref = client.dataset(google_biquery_dataset)
table_ref = dataset_ref.table("data")
table = client.get_table(table_ref)  # API Request

# View table properties
schema = table.schema
print(schema)

[SchemaField('analysis_sample_rate', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('artist_7digitalid', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('artist_familiarity', 'FLOAT', 'NULLABLE', None, (), None), SchemaField('artist_hotttnesss', 'FLOAT', 'NULLABLE', None, (), None), SchemaField('artist_id', 'STRING', 'NULLABLE', None, (), None), SchemaField('artist_latitude', 'FLOAT', 'NULLABLE', None, (), None), SchemaField('artist_location', 'STRING', 'NULLABLE', None, (), None), SchemaField('artist_longitude', 'FLOAT', 'NULLABLE', None, (), None), SchemaField('artist_mbid', 'STRING', 'NULLABLE', None, (), None), SchemaField('artist_mbtags', 'RECORD', 'NULLABLE', None, (SchemaField('list', 'RECORD', 'REPEATED', None, (SchemaField('element', 'STRING', 'NULLABLE', None, (), None),), None),), None), SchemaField('artist_mbtags_count', 'RECORD', 'NULLABLE', None, (SchemaField('list', 'RECORD', 'REPEATED', None, (SchemaField('element', 'INTEGER', 'NULLABLE', None, (), None),), Non

In [20]:


table_id = f"{google_project_id}.{google_biquery_dataset}.data_partitioned"

table = bigquery.Table(table_id, schema=schema)
table.range_partitioning = bigquery.RangePartitioning(
    # To use integer range partitioning, select a top-level REQUIRED /
    # NULLABLE column with INTEGER / INT64 data type.
    field="year",
    range_=bigquery.PartitionRange(start=0, end=2200, interval=10),
)

table.clustering_fields = ["tempo_int", "mode", "key"]

# TODO(developer): Set table_id to the ID of the destination table.
# table_id = "your-project.your_dataset.your_table_name"


table = client.create_table(table)  # Make an API request.
print(
    "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
)

Created table alpine-freedom-344703.million_song.data_partitioned


In [24]:
df_to_save.write \
        .mode('append') \
        .format("bigquery") \
        .option("temporaryGcsBucket",google_cloud_storage_id) \
        .option('table', f"{google_biquery_dataset}.data_partitioned") \
        .save()